In [1]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [64]:
glove = joblib.load('/home/jeremy/GloVe-1.2/glove.840B.300d.dict.pckl')

In [63]:
train_clean = joblib.load('data/train_clean.pckl')

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [65]:
train_clean.head().T

,0,1,2,3,4
id,0000997932d777bf,000103f0d9cfb60f,000113f07ec002fd,0001b41b1c6bb37e,0001d958c54c6e35
comment_text,Explanation\nWhy the edits made under my usern...,D'aww! He matches this background colour I'm s...,"Hey man, I'm really not trying to edit war. It...","""\nMore\nI can't make any real suggestions on ...","You, sir, are my hero. Any chance you remember..."
toxic,0,0,0,0,0
severe_toxic,0,0,0,0,0
obscene,0,0,0,0,0
threat,0,0,0,0,0
insult,0,0,0,0,0
identity_hate,0,0,0,0,0
clean_text_list,"[explanation, edits, make, username, hardcore,...","[aww, match, background, colour, seemingly, st...","[hey, man, really, try, edit, war, guy, consta...","[make, real, suggestion, improvement, wonder, ...","[sir, hero, chance, remember, page]"
clean_text,explanation edits make username hardcore metal...,aww match background colour seemingly stick th...,hey man really try edit war guy constantly rem...,make real suggestion improvement wonder sectio...,sir hero chance remember page


In [66]:
impute = np.zeros(300)

In [87]:
def sentence_to_tensor(sentence):
    g = [ glove.get(w, impute) for w in sentence ]
    if len(g)==0:
        g = [ impute ]
    return torch.FloatTensor(np.asarray(g))

In [88]:
# create list of (tensor, label) tuples

tensor_list = [ sentence_to_tensor(x) for x in train_clean.clean_text_list ]

In [89]:
tensor_labels = [ torch.LongTensor(x) for x in train_clean.toxic ]

In [102]:
data_list = list(zip(tensor_list, train_clean.toxic))

In [97]:
t = data_list[0][0]

In [98]:
t.shape

torch.Size([27, 300])

In [101]:
pad_tensor(t, 28, 0).shape

torch.Size([28, 300])

In [59]:
# these functions allow a dataloader to process inputs of various lengths 
# and zero pad each batch to max length within that batch

# input to dataloader should be a list of tuples (tensor, label)

def pad_tensor(vec, pad, dim):
    """
    args:
        vec - tensor to pad
        pad - the size to pad to
        dim - dimension to pad

    return:
        a new tensor padded to 'pad' in dimension 'dim'
    """
    pad_size = list(vec.shape)
    pad_size[dim] = pad - vec.size(dim)
    return torch.cat([vec, torch.zeros(*pad_size).type_as(vec)], dim=dim)


class PadCollate:
    """
    a variant of collate_fn that pads according to the longest sequence in
    a batch of sequences
    """

    def __init__(self, dim=0):
        """
        args:
            dim - the dimension to be padded (dimension of time in sequences)
        """
        self.dim = dim

    def pad_collate(self, batch):
        """
        args:
            batch - list of (tensor, label)

        reutrn:
            xs - a tensor of all examples in 'batch' after padding
            ys - a LongTensor of all labels in batch
        """
        # find longest sequence
        max_len = max(map(lambda x: x[0].shape[self.dim], batch))
        print(max_len)
        
        # pad according to max_len
        batch = list(map(lambda x: (pad_tensor(x[0], pad=max_len, dim=self.dim), x[1]), batch))
        
        # stack all
        xs = torch.stack(list(map(lambda x: x[0], batch)), dim=0)
        ys = torch.LongTensor(list(map(lambda x: x[1], batch)))
        return xs, ys

    def __call__(self, batch):
        return self.pad_collate(batch)

In [111]:
train_loader = DataLoader(data_list, batch_size=5, collate_fn=PadCollate(dim=0))

In [7]:
# from Shawn1993 on Github

class CNN_Text(nn.Module):
    
    def __init__(self, args):
        super(CNN_Text, self).__init__()
        self.args = args
        
       """ V = args.embed_num      # vocab size ... this vesion tunes vectors (will change and use glove)
        D = args.embed_dim      # embedding dimension
        C = args.class_num      # outcome classes (e.g. 2 for binary)
        Ci = 1                  # input channel is 1, compared to images wich may have 3 (r,g,b)
        Co = args.kernel_num    # number of filters per region emsize
        Ks = args.kernel_sizes  # list of filter region sizes
"""
    
        #V = args.embed_num      # vocab size ... this vesion tunes vectors (will change and use glove)
        D = 300      # embedding dimension
        C = 2      # outcome classes (e.g. 2 for binary)
        Ci = 1      # input channel is 1 for text, compared to images wich may have 3 (r,g,b)
        Co = 100    # number of filters per region emsize
        Ks = [3, 5]  # list of filter region sizes

        #self.embed = nn.Embedding(V, D)
        # self.convs1 = [nn.Conv2d(Ci, Co, (K, D)) for K in Ks]
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])  # holds list of modules for each region size
        '''
        self.conv13 = nn.Conv2d(Ci, Co, (3, D))
        self.conv14 = nn.Conv2d(Ci, Co, (4, D))
        self.conv15 = nn.Conv2d(Ci, Co, (5, D))
        '''
        self.dropout = nn.Dropout(.5)
        self.fc1 = nn.Linear(len(Ks)*Co, C)

    """
    # is this func even needed?  
    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3)  # (N, Co, W)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x
    """
    
    def forward(self, x):
        x = self.embed(x)  # (N, W, D)  prob remove this for pre-trained glove vectors?
        
        if self.args.static:
            x = Variable(x)

        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        '''
        x1 = self.conv_and_pool(x,self.conv13) #(N,Co)
        x2 = self.conv_and_pool(x,self.conv14) #(N,Co)
        x3 = self.conv_and_pool(x,self.conv15) #(N,Co)
        x = torch.cat((x1, x2, x3), 1) # (N,len(Ks)*Co)
        '''
        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc1(x)  # (N, C)
        return logit
